In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd
load_dotenv()
import requests
import psycopg2
from tqdm import tqdm
pd.set_option('display.max_colwidth', None)

links_dict = {}

In [ ]:
def create_session_token():
    url = "https://api.voluum.com/auth/access/session"

    payload = {
        "accessId": os.getenv("VOLUUM_ACCESS_ID"),
        "accessKey": os.getenv("VOLUUM_ACCESS_KEY")
    }

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "Accept": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)

    return response.json()["token"]

access_token = create_session_token()

def get_broadcasts(date):
    url = "https://sms.messagewhiz.com/api/3/Broadcast/List"

    start = 0  # Initialize starting point for pagination
    total_fetched = 0 
    list_of_broadcasts = []
    
    global_check = True
    while True:
        params = {
            'limit': 100,
            'start': start,
        }

        headers = {
            "apikey": "placeholder",
            "Content-Type": "application/json"
        }

        # Send the request
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Failed to retrieve data: {response.status_code}")
            break   

        broadcast_json = response.json()
        broadcast_res = broadcast_json["result"]
        count = len(broadcast_res)  # Count the number of items fetched in this iteration
        # Process each item
        for item in broadcast_res:
            if date in item["send_date"]:
                list_of_broadcasts.append(item)
            else:
                if f"2026-02-06" in item["send_date"]:
                    continue
                global_check = False           
                break

        total_fetched += count

        if not global_check:
            break

        if count < 100:  # If less than 100 items, stop fetching
            break

        start += count
    
    return list_of_broadcasts

def get_links():
    url = "https://sms.messagewhiz.com/api/3/Link"

    start = 0  # Initialize starting point for pagination
    total_fetched = 0 
    list_of_links = []
    
    global_check = True
    while True:
        params = {
            'limit': 100,
            'start': start,
        }

        headers = {
            "apikey": "placeholder",
            "Content-Type": "application/json"
        }

        # Send the request
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Failed to retrieve data: {response.status_code}")
            break   

        broadcast_json = response.json()
        broadcast_res = broadcast_json["result"]
        count = len(broadcast_res)  # Count the number of items fetched in this iteration
        print(broadcast_res)
        # Process each item
        for item in broadcast_res:
            list_of_links.append(item)

        total_fetched += count
        print(f"Total fetched so far: {total_fetched}")

        if not global_check:
            break

        if count < 100:  # If less than 100 items, stop fetching
            break

        start += count
    
    return list_of_links

def extract_link_id(message):
    return message.split("{{link:")[1].split("}}")[0]

def get_url(link_id):
    return links_dict.get(link_id, None)

def get_campaign_id(link):
    return link.split("m/")[1] if link else None

def format_real_price(price):
    return price/ 100

In [9]:
date = "2026-02-05" 
broadcasts = get_broadcasts(date)
f"Number of broadcasts fetched for date: {date} are {len(broadcasts)}"

'Number of broadcasts fetched for date: 2026-02-05 are 1236'

In [10]:
links = get_links()
links_dict = {str(link['id']): link["url"].split("?")[0] for link in links}
f"Number of links fetched are {len(links_dict)}"

[{'enabled': True, 'platform_based_links': None, 'id': 7006012, 'name': 'SpinPanda NL CONV', 'url': 'https://trk.lxryspin.com/6439867f-c122-4c1f-b80a-ed4ed89012ae', 'type': 0, 'expiration_time': None, 'expiration_url': None, 'created_at': '2026-02-05T22:59:13.000Z', 'updated_at': '2026-02-05T22:59:13.000Z', 'updated_by': 'growthpartners01@proton.me', 'created_by': 'growthpartners01@proton.me', 'department_id': 15769}, {'enabled': True, 'platform_based_links': None, 'id': 7006009, 'name': 'PiperSpin ES Clickers', 'url': 'https://trk.lxryspin.com/d9ae4193-803c-48ce-b29b-02590e286ca7?Phone={recipients.id}', 'type': 1, 'expiration_time': None, 'expiration_url': None, 'created_at': '2026-02-05T20:09:08.000Z', 'updated_at': '2026-02-05T20:09:08.000Z', 'updated_by': 'growthpartners01@proton.me', 'created_by': 'growthpartners01@proton.me', 'department_id': 15769}, {'enabled': True, 'platform_based_links': None, 'id': 7006008, 'name': 'GodOfWins DK', 'url': 'https://tr.stockholmlab.com/d0df9fe0

'Number of links fetched are 1143'

In [11]:
broadcast_df = pd.DataFrame(broadcasts)
broadcast_df["link_id"] = broadcast_df["message_body"].apply(extract_link_id)
broadcast_df["url"] = broadcast_df["link_id"].apply(get_url)
broadcast_df["campaign_id"] = broadcast_df["url"].apply(get_campaign_id)
broadcast_df["real_price"] = broadcast_df.real_price.apply(format_real_price)
broadcast_df[:1]

,id,user_id,name,type,message_body,enabled,trigger_id,send_date,state,parent_id,...,created_at,created_by,updated_by,department_id,department,dlr,broadcastConversion,link_id,url,campaign_id
0,395873043,15696,NL_CONV_19JAN_3.7K_chunk#13.csv,11,"Gefeliciteerd! We hebben 3000 EUR en 225 G S aan je account toegevoegd, plus een speciale bonus {{link:7006012}}",1,None,2026-02-05T23:25:00.000Z,4,None,...,2026-02-05T23:03:34.000Z,api+15769@15696.mmd,None,15769,"{'id': 15769, 'name': 'Team'}","{'broadcast_id': 395873043, 'sent_count': 7, 'delivered_count': 208, 'undelivered_count': 2, 'rejected_count': 0, 'expired_count': 0, 'failed_count': 0, 'read_count': 0, 'no_balance_count': None}","{'uniqueClicks': 9, 'totalClicks': 11, 'recipientCount': 217, 'conversion': '4.15', 'broadcastId': 395873043, 'segmented': True}",7006012,https://trk.lxryspin.com/6439867f-c122-4c1f-b80a-ed4ed89012ae,6439867f-c122-4c1f-b80a-ed4ed89012ae


In [108]:
import json
json.loads(broadcast_df[:1].to_json())

{'id': {'0': 395328565},
 'user_id': {'0': 15696},
 'name': {'0': 'IT_51k_27_11_25_Iphone_OK1_chunk#74.csv'},
 'type': {'0': 11},
 'message_body': {'0': 'Il tuo conto e stato accreditato con un Bonus Elite, 1000 EUR in denaro reale, 100TG Sbloccalo qui {{link:6873306}}'},
 'enabled': {'0': 1},
 'trigger_id': {'0': None},
 'send_date': {'0': '2026-01-16T23:45:00.000Z'},
 'state': {'0': 4},
 'parent_id': {'0': None},
 'real_price': {'0': 0.9801},
 'estimated_price': {'0': 98.01},
 'create_date': {'0': '2026-01-16T21:19:10.000Z'},
 'estimated_count': {'0': 27},
 'utc_offset': {'0': None},
 'recipient_count': {'0': 27},
 'next_segment_size': {'0': None},
 'eb_type': {'0': None},
 'company_name': {'0': 'FINAL DISTRICT L.L.C-FZ (One Stop Fze) '},
 'timezone': {'0': None},
 'send_now': {'0': False},
 'updated_at': {'0': None},
 'created_at': {'0': '2026-01-16T21:19:10.000Z'},
 'created_by': {'0': 'api+15769@15696.mmd'},
 'updated_by': {'0': None},
 'department_id': {'0': 15769},
 'department'

In [12]:
def get_campaign_data(campaign_id, from_date, to_date):
    url = f"https://panel-api2.voluum.com/report?reportType=table&limit=500&dateRange=yesterday&from={from_date}&to={to_date}&searchMode=TEXT&currency=EUR&sort=visits&direction=ASC&reportDataType=0&offset=0&groupBy=custom-variable-1&groupBy=ip&groupBy=browser-version&groupBy=os-version&column=profit&column=customVariable1&column=visits&column=uniqueVisits&column=suspiciousVisitsPercentage&column=campaignNoConversionsWarning&column=conversions&column=costSources&column=cost&column=revenue&column=roi&column=cv&column=epv&column=cpv&column=errors&column=Click2Reg&column=Reg2FTD&column=customConversions1&column=customRevenue1&column=customConversions2&column=customRevenue2&column=customConversions3&column=customRevenue3&column=osVersion&column=browserVersion&column=actions&column=type&column=clicks&column=suspiciousClicksPercentage&column=suspiciousVisits&column=suspiciousClicks&tz=Etc/GMT&filter1=campaign&filter1Value={campaign_id}&tz=Utc"
    headers = {
        "cwauth-token": access_token
    }
    try:
        response = requests.get(url, headers=headers)
        campaigns = response.json()
    except Exception as e:
        print(f"Error fetching campaign data for campaign ID {campaign_id}: {e}")
        raise e
    return campaigns

In [75]:
broadcast_df.iloc[0]["send_date"]

'2026-01-16T23:45:00.000Z'

In [ ]:
def get_broadcasts():
    url = "https://sms.messagewhiz.com/api/3/Broadcast/List"

    start = 0  # Initialize starting point for pagination
    total_fetched = 0 
    list_of_broadcasts = []
    
    global_check = True
    while True:
        params = {
            'limit': 100,
            'start': start,
        }

        headers = {
            "apikey": "placeholder",
            "Content-Type": "application/json"
        }

        # Send the request
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Failed to retrieve data: {response.status_code}")
            break   

        broadcast_json = response.json()
        broadcast_res = broadcast_json["result"]
        count = len(broadcast_res)  # Count the number of items fetched in this iteration
        # Process each item
        for item in broadcast_res:
            if "2025-12-31" in item["send_date"]:
                global_check = False           
                break
            if "2026-01-17" in item["send_date"]:
                continue
            list_of_broadcasts.append(item)
        total_fetched += count

        if not global_check:
            break
        print(f"Total fetched so far: {total_fetched}, Date of last item: {broadcast_res[-1]['send_date']}")
        if count < 100:  # If less than 100 items, stop fetching
            break

        start += count
    
    return list_of_broadcasts

In [5]:
broadcasts_history = get_broadcasts()
len(broadcasts_history)

Total fetched so far: 100, Date of last item: 2026-01-17T18:53:00.000Z
Total fetched so far: 200, Date of last item: 2026-01-17T16:41:00.000Z
Total fetched so far: 300, Date of last item: 2026-01-17T12:03:00.000Z
Total fetched so far: 400, Date of last item: 2026-01-17T11:17:00.000Z
Total fetched so far: 500, Date of last item: 2026-01-16T21:49:00.000Z
Total fetched so far: 600, Date of last item: 2026-01-16T16:45:00.000Z
Total fetched so far: 700, Date of last item: 2026-01-16T15:25:00.000Z
Total fetched so far: 800, Date of last item: 2026-01-16T15:37:00.000Z
Total fetched so far: 900, Date of last item: 2026-01-16T13:45:00.000Z
Total fetched so far: 1000, Date of last item: 2026-01-16T13:33:00.000Z
Total fetched so far: 1100, Date of last item: 2026-01-16T12:19:00.000Z
Total fetched so far: 1200, Date of last item: 2026-01-15T21:43:00.000Z
Total fetched so far: 1300, Date of last item: 2026-01-15T19:15:00.000Z
Total fetched so far: 1400, Date of last item: 2026-01-15T17:21:00.000Z
T

9933

In [6]:
broadcasts_history_df = pd.DataFrame(broadcasts_history)
broadcasts_history_df["link_id"] = broadcasts_history_df["message_body"].apply(extract_link_id)
broadcasts_history_df["url"] = broadcasts_history_df["link_id"].apply(get_url)
broadcasts_history_df["campaign_id"] = broadcasts_history_df["url"].apply(get_campaign_id)
broadcasts_history_df.head()

,id,user_id,name,type,message_body,enabled,trigger_id,send_date,state,parent_id,...,created_at,created_by,updated_by,department_id,department,dlr,broadcastConversion,link_id,url,campaign_id
0,395328565,15696,IT_51k_27_11_25_Iphone_OK1_chunk#74.csv,11,"Il tuo conto e stato accreditato con un Bonus Elite, 1000 EUR in denaro reale, 100TG Sbloccalo qui {{link:6873306}}",1,None,2026-01-16T23:45:00.000Z,4,None,...,2026-01-16T21:19:10.000Z,api+15769@15696.mmd,None,15769,"{'id': 15769, 'name': 'Team'}","{'broadcast_id': 395328565, 'sent_count': 2, 'delivered_count': 25, 'undelivered_count': 0, 'rejected_count': 0, 'expired_count': 0, 'failed_count': 0, 'read_count': 0}","{'uniqueClicks': 0, 'totalClicks': 0, 'recipientCount': 27, 'conversion': '0.00', 'broadcastId': 395328565, 'segmented': True}",6873306,https://trk.metripartners.com/ae71eafc-5d9d-4b82-9519-cd2944d84583,ae71eafc-5d9d-4b82-9519-cd2944d84583
1,395328564,15696,IT_51k_27_11_25_Iphone_OK1_chunk#73.csv,11,"Il tuo conto e stato accreditato con un Bonus Elite, 1000 EUR in denaro reale, 100TG Sbloccalo qui {{link:6873306}}",1,None,2026-01-16T23:43:00.000Z,4,None,...,2026-01-16T21:19:09.000Z,api+15769@15696.mmd,None,15769,"{'id': 15769, 'name': 'Team'}","{'broadcast_id': 395328564, 'sent_count': 27, 'delivered_count': 465, 'undelivered_count': 6, 'rejected_count': 0, 'expired_count': 0, 'failed_count': 0, 'read_count': 0}","{'uniqueClicks': 17, 'totalClicks': 23, 'recipientCount': 498, 'conversion': '3.41', 'broadcastId': 395328564, 'segmented': True}",6873306,https://trk.metripartners.com/ae71eafc-5d9d-4b82-9519-cd2944d84583,ae71eafc-5d9d-4b82-9519-cd2944d84583
2,395328563,15696,IT_51k_27_11_25_Iphone_OK1_chunk#72.csv,11,"Il tuo conto e stato accreditato con un Bonus Elite, 1000 EUR in denaro reale, 100TG Sbloccalo qui {{link:6873306}}",1,None,2026-01-16T23:41:00.000Z,4,None,...,2026-01-16T21:19:09.000Z,api+15769@15696.mmd,None,15769,"{'id': 15769, 'name': 'Team'}","{'broadcast_id': 395328563, 'sent_count': 25, 'delivered_count': 469, 'undelivered_count': 4, 'rejected_count': 0, 'expired_count': 0, 'failed_count': 0, 'read_count': 0}","{'uniqueClicks': 12, 'totalClicks': 12, 'recipientCount': 498, 'conversion': '2.41', 'broadcastId': 395328563, 'segmented': True}",6873306,https://trk.metripartners.com/ae71eafc-5d9d-4b82-9519-cd2944d84583,ae71eafc-5d9d-4b82-9519-cd2944d84583
3,395328562,15696,IT_51k_27_11_25_Iphone_OK1_chunk#71.csv,11,"Il tuo conto e stato accreditato con un Bonus Elite, 1000 EUR in denaro reale, 100TG Sbloccalo qui {{link:6873306}}",1,None,2026-01-16T23:39:00.000Z,4,None,...,2026-01-16T21:19:08.000Z,api+15769@15696.mmd,None,15769,"{'id': 15769, 'name': 'Team'}","{'broadcast_id': 395328562, 'sent_count': 33, 'delivered_count': 457, 'undelivered_count': 8, 'rejected_count': 0, 'expired_count': 0, 'failed_count': 0, 'read_count': 0}","{'uniqueClicks': 7, 'totalClicks': 8, 'recipientCount': 498, 'conversion': '1.41', 'broadcastId': 395328562, 'segmented': True}",6873306,https://trk.metripartners.com/ae71eafc-5d9d-4b82-9519-cd2944d84583,ae71eafc-5d9d-4b82-9519-cd2944d84583
4,395328561,15696,IT_51k_27_11_25_Iphone_OK1_chunk#70.csv,11,"Il tuo conto e stato accreditato con un Bonus Elite, 1000 EUR in denaro reale, 100TG Sbloccalo qui {{link:6873306}}",1,None,2026-01-16T23:37:00.000Z,4,None,...,2026-01-16T21:19:07.000Z,api+15769@15696.mmd,None,15769,"{'id': 15769, 'name': 'Team'}","{'broadcast_id': 395328561, 'sent_count': 35, 'delivered_count': 454, 'undelivered_count': 9, 'rejected_count': 0, 'expired_count': 0, 'failed_count': 0, 'read_count': 0}","{'uniqueClicks': 5, 'totalClicks': 5, 'recipientCount': 498, 'conversion': '1.00', 'broadcastId': 395328561, 'segmented': True}",6873306,https://trk.metripartners.com/ae71eafc-5d9d-4b82-9519-cd2944d84583,ae71eafc-5d9d-4b82-9519-cd2944d84583


In [15]:
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASSWORD")

connection = psycopg2.connect(
            host=db_host,
            database=db_name,
            user=db_user,
            password=db_pass
)
cursor = connection.cursor()

In [7]:
rows_to_insert = []

def add_broadcast_campaign_data():
    for _, row in tqdm(broadcasts_history_df[broadcasts_history_df["state"] == 4].iterrows(), total=len(broadcasts_history_df[broadcasts_history_df["state"] == 4])):
        # ---- Parse send_date safely ----
        date, time = row["send_date"].split('T')
        from_date = date + 'T' + time.split(':')[0] + ':00:00.000Z'
        to_date = f'2026-01-{(int(date.split('-')[-1]) + 1)}T00:00:00.000Z'

        try: 
            totals = get_campaign_data(
                row["campaign_id"],
                from_date=str(from_date),
                to_date=str(to_date)
            )
            totals = totals["totals"]
        except Exception as e:
            print(f"Error fetching campaign data for campaign ID {row['campaign_id']}: {e}, {totals}")
            raise e
        rows_to_insert.append({
            # Core identifiers
            "id": row["id"],
            "broadcast_id": row["id"],
            "user_id": row["user_id"],
            "department_id": row.get("department", {}).get("id"),
            "department_name": row.get("department", {}).get("name"),

            "campaign_id": row["campaign_id"],
            "link_id": row["link_id"],
            "url": row["url"],

            # Broadcast metadata
            "name": row["name"],
            "type": row["type"],
            "message_body": row["message_body"],
            "enabled": bool(row["enabled"]),
            "trigger_id": row["trigger_id"],
            "parent_id": row["parent_id"],
            "state": row["state"],
            "eb_type": row["eb_type"],
            "company_name": row["company_name"],
            "timezone": row["timezone"],
            "utc_offset": row["utc_offset"],
            "send_now": row["send_now"],

            # Dates
            "send_date": row["send_date"],
            "create_date": pd.to_datetime(row["create_date"], utc=True),
            "created_at": pd.to_datetime(row["created_at"], utc=True),
            "updated_at": pd.to_datetime(row["updated_at"], utc=True),

            "created_by": row["created_by"],
            "updated_by": row["updated_by"],

            # Pricing & counts
            "real_price": row["real_price"],
            "estimated_price": row["estimated_price"],
            "estimated_count": row["estimated_count"],
            "recipient_count": row["recipient_count"],
            "next_segment_size": row["next_segment_size"],

            # DLR stats
            "dlr_sent_count": row["dlr"]["sent_count"] if row.get("dlr") else 0,
            "dlr_delivered_count": row["dlr"]["delivered_count"] if row.get("dlr") else 0,
            "dlr_undelivered_count": row["dlr"]["undelivered_count"] if row.get("dlr") else 0,
            "dlr_rejected_count": row["dlr"]["rejected_count"] if row.get("dlr") else 0,
            "dlr_expired_count": row["dlr"]["expired_count"] if row.get("dlr") else 0,
            "dlr_failed_count": row["dlr"]["failed_count"] if row.get("dlr") else 0,
            "dlr_read_count": row["dlr"]["read_count"] if row.get("dlr") else 0,

            # Broadcast conversion
            "bc_unique_clicks": row["broadcastConversion"]["uniqueClicks"],
            "bc_total_clicks": row["broadcastConversion"]["totalClicks"],
            "bc_recipient_count": row["broadcastConversion"]["recipientCount"],
            "bc_conversion": float(row["broadcastConversion"]["conversion"]),
            "bc_segmented": row["broadcastConversion"]["segmented"],

            # Performance metrics (from totals)
            "click_2_reg": totals["Click2Reg"],
            "reg_2_ftd": totals["Reg2FTD"],
            "clicks": totals["clicks"],
            "conversions": totals["conversions"],
            "cost": totals["cost"],
            "cost_sources": totals["costSources"],
            "cpv": totals["cpv"],

            "custom_conversions_1": totals["customConversions1"],
            "custom_conversions_2": totals["customConversions2"],
            "custom_conversions_3": totals["customConversions3"],

            "custom_revenue_1": totals["customRevenue1"],
            "custom_revenue_2": totals["customRevenue2"],
            "custom_revenue_3": totals["customRevenue3"],

            "cv": totals["cv"],
            "epv": totals["epv"],
            "errors": totals["errors"],
            "profit": totals["profit"],
            "revenue": totals["revenue"],
            "roi": totals["roi"],

            "suspicious_clicks": totals["suspiciousClicks"],
            "suspicious_clicks_percentage": totals["suspiciousClicksPercentage"],
            "suspicious_visits": totals["suspiciousVisits"],
            "suspicious_visits_percentage": totals["suspiciousVisitsPercentage"],

            "unique_visits": totals["uniqueVisits"],
            "visits": totals["visits"],
        })

In [10]:
add_broadcast_campaign_data()
len(rows_to_insert)

100%|██████████| 9421/9421 [1:45:49<00:00,  1.48it/s]  


9421

In [11]:
pd.DataFrame(rows_to_insert).to_csv("broadcast_campaign_data.csv", index=False)

In [16]:
from psycopg2.extras import execute_values

columns = rows_to_insert[0].keys()

query = f"""
INSERT INTO public.api_broadcast_campaign_stats ({",".join(columns)})
VALUES %s
"""

values = [
    tuple(row[col] for col in columns)
    for row in rows_to_insert
]

execute_values(cursor, query, values)
connection.commit()

In [2]:
from app.utils.mmd_data_handler import MMDDataHandler
import pandas as pd
mmd_data_handler = MMDDataHandler()

In [3]:
broadcasts = mmd_data_handler.get_broadcasts("2026-01-20")
mmd_data_handler.get_links()
broadcast_df = pd.DataFrame(broadcasts)
broadcast_df["link_id"] = broadcast_df["message_body"].apply(lambda x: x.split("{{link:")[1].split("}}")[0])
broadcast_df["url"] = broadcast_df["link_id"].apply(lambda x: mmd_data_handler.links_dict.get(int(x), None))
broadcast_df["campaign_id"] = broadcast_df["url"].apply(lambda x: x.split("m/")[1] if x else None)
broadcast_df["real_price"] = broadcast_df["real_price"].apply(lambda price: price / 100)
print(f"Fetched {len(broadcasts)} rows.")

Fetched 4356 rows.


In [18]:
total_rows = []

In [24]:
from datetime import timezone, timedelta

grouped_rows = {}
for _, row in tqdm(broadcast_df.iterrows(), total=len(broadcast_df)):
    date, time = row["send_date"].split("T")
    hour = int(time.split(":")[0])

    from_dt = datetime.strptime(
        f"{date} {hour:02d}:00:00",
        "%Y-%m-%d %H:%M:%S"
    ).replace(tzinfo=timezone.utc)

    # Current time rounded down to hour
    now = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)

    # Decide to_date
    if now - from_dt > timedelta(hours=24):
        to_dt = from_dt + timedelta(hours=24)
    else:
        to_dt = now

    # Format back to string
    from_date = from_dt.strftime("%Y-%m-%dT%H:00:00.000Z")
    to_date = to_dt.strftime("%Y-%m-%dT%H:00:00.000Z")
    if (from_date, to_date, row["campaign_id"]) not in grouped_rows:
        grouped_rows[from_date, to_date, row["campaign_id"]] = mmd_data_handler.rate_limited_get_campaign(
                                                                row["campaign_id"],
                                                                from_date=from_date,
                                                                to_date=to_date
                                                            )["totals"]

 62%|██████▏   | 2693/4356 [02:51<03:59,  6.96it/s]

Error fetching campaign data for campaign ID fc8001d6-6ea7-4e2e-a3e8-f86830574969: Expecting value: line 1 column 1 (char 0)


100%|██████████| 4356/4356 [04:34<00:00, 15.86it/s]


In [46]:
grouped_rows[('2026-01-23T23:00:00.000Z',
  '2026-01-24T11:00:00.000Z',
  'bdf116b4-0aed-450a-9ce2-72607c330ef1')]

{'Click2Reg': 20.16129,
 'Reg2FTD': 0.0,
 'clicks': 0,
 'conversions': 0,
 'cost': 0.0,
 'costSources': ['MANUAL_UPDATE'],
 'cpv': 0.0,
 'customConversions1': 25,
 'customConversions2': 0,
 'customConversions3': 0,
 'customRevenue1': 0.0,
 'customRevenue2': 0.0,
 'customRevenue3': 0.0,
 'cv': 0.0,
 'epv': 0.0,
 'errors': 0,
 'profit': 0.0,
 'revenue': 0.0,
 'roi': 0.0,
 'suspiciousClicks': 0,
 'suspiciousClicksPercentage': 0.0,
 'suspiciousVisits': 0,
 'suspiciousVisitsPercentage': 0.0,
 'uniqueVisits': 124,
 'visits': 137}

In [ ]:
total_rows = []
for _, row in tqdm(broadcast_df.iterrows(), total=len(broadcast_df)):
    if row["state"] in [0, 2]:
        continue
    try:
        date, time = row["send_date"].split("T")
        hour = int(time.split(":")[0])

        from_dt = datetime.strptime(
            f"{date} {hour:02d}:00:00",
            "%Y-%m-%d %H:%M:%S"
        ).replace(tzinfo=timezone.utc)

        # Current time rounded down to hour
        now = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)

        # Decide to_date
        if now - from_dt > timedelta(hours=24):
            to_dt = from_dt + timedelta(hours=24)
        else:
            to_dt = now

        # Format back to string
        from_date = from_dt.strftime("%Y-%m-%dT%H:00:00.000Z")
        to_date = to_dt.strftime("%Y-%m-%dT%H:00:00.000Z")
        totals = grouped_rows[from_date, to_date, row["campaign_id"]]
        total_rows.append({
            "Campaign ID": row["campaign_id"],
            "Broadcast ID": row["id"],
            "Name": row["name"],
            "Send Date": row["send_date"],

            # "Real Price": round(row["real_price"], 2),
            "Estimated Price": round(row["estimated_price"] / 100, 2),
            "Recipient Count": row["recipient_count"],

            "BC Unique Clicks": row["broadcastConversion"]["uniqueClicks"],
            "BC Total Clicks": row["broadcastConversion"]["totalClicks"],
            # "BC Recipient Count": row["broadcastConversion"]["recipientCount"],
            "CTR": f'{round(float(row["broadcastConversion"]["conversion"]), 2)}%',

            "DLR Delivered Count": row["dlr"]["delivered_count"] if row.get("dlr") else 0,
            "DLR Sent Count": row["dlr"]["sent_count"] if row.get("dlr") else 0,
            "DLR Undelivered Count": row["dlr"]["undelivered_count"] if row.get("dlr") else 0,
            "DLR Rejected Count": row["dlr"]["rejected_count"] if row.get("dlr") else 0,
            # "DLR Expired Count": row["dlr"]["expired_count"] if row.get("dlr") else 0,
            # "DLR Failed Count": row["dlr"]["failed_count"] if row.get("dlr") else 0,
            # "DLR Read Count": row["dlr"]["read_count"] if row.get("dlr") else 0,


            "Voluum Visits": totals["visits"],
            "Voluum Unique Visits": totals["uniqueVisits"],
            "Click 2 REG": f'{round(totals["Click2Reg"], 2)}%',
            "Reg 2 FTD": f'{totals["Reg2FTD"]}%',
            # "Clicks": totals["clicks"],
            # "Conversions": totals["conversions"],
            # "revenue": totals["revenue"],

            "message_body": row["message_body"],
        })
    except Exception as e:
        print(_, row.to_json())
        raise e
    # print(total_rows)
    # break
# total_rows_df = pd.DataFrame(total_rows)
# total_rows_df.to_csv("MMD_Data_20-23_Jan.csv")

100%|██████████| 4356/4356 [00:00<00:00, 24508.38it/s]


In [29]:
total_rows = pd.DataFrame(total_rows)
total_rows.to_csv("MMD_Data_20-23_Jan.csv")

In [43]:
total_rows.head

<bound method NDFrame.head of       Broadcast ID                                           Name  \
0        395459375  ES_IOSCLK_SendingOK2_23012026HLR_chunk#28.csv   
1        395459373  ES_IOSCLK_SendingOK2_23012026HLR_chunk#27.csv   
2        395459371  ES_IOSCLK_SendingOK2_23012026HLR_chunk#26.csv   
3        395459369  ES_IOSCLK_SendingOK2_23012026HLR_chunk#25.csv   
4        395459367  ES_IOSCLK_SendingOK2_23012026HLR_chunk#24.csv   
...            ...                                            ...   
4018     395378802           DE_38k_01_12_25_LV_3to8k_chunk#4.csv   
4019     395378801           DE_38k_01_12_25_LV_3to8k_chunk#3.csv   
4020     395378799           DE_38k_01_12_25_LV_3to8k_chunk#2.csv   
4021     395378798           DE_38k_01_12_25_LV_3to8k_chunk#1.csv   
4022     395374744                    IT_MC_Dec_2025_chunk#52.csv   

                     Send Date  Estimated Price  Recipient Count  \
0     2026-01-23T23:29:00.000Z             5.83              175   
1    

## Adding encryption to given csv lists for testing purpose

In [31]:
from app.utils.encryption_handler import EncryptionHandler
import pandas as pd

encryption_handler = EncryptionHandler()

In [39]:
file_path = "/Users/zohaibmunir/Downloads/NO_CLK_new911kmf_encrypt"
list_df = pd.read_csv(f"{file_path}.csv")

list_df

,phone,source
0,4740000014,no_clk_new911kmf
1,4740018913,no_clk_new911kmf
2,4740037872,no_clk_new911kmf
3,4740101456,no_clk_new911kmf
4,4740106857,no_clk_new911kmf
...,...,...
18441,4799720518,no_clk_new911kmf
18442,4799722054,no_clk_new911kmf
18443,4799723999,no_clk_new911kmf
18444,4799724521,no_clk_new911kmf


In [41]:
list_df["encrypted_phone"] = list_df.phone.astype(str).apply(encryption_handler.encrypt_number)
list_df.head()

,phone,source,encrypted_phone
0,4740000014,no_clk_new911kmf,b2hxdUMFn1yni7R_PwhFYCViuhXTyyNbzB8bUPXwCN9VfSLoWQ4
1,4740018913,no_clk_new911kmf,UdbREMmtJP_FE05NM_PsCF1OO8zvqAxET4VN8jY3HArJDstuIXQ
2,4740037872,no_clk_new911kmf,dmybyddqUcE1qUR222bugEYc7U-hjXJYP3ZnxirhE4WVlBf4Swo
3,4740101456,no_clk_new911kmf,d_MrJ7knpJ-i3PM69voXgtDgTGzPJ0PFidj9OlqQIHfluByMtbs
4,4740106857,no_clk_new911kmf,UHvfKDrbhJ8cmV7AdVb5hekx7lCmIiq09JIuxoDrP-W-egk8Oyo


In [42]:
list_df.to_csv(f"{file_path}_ENCRYPTED.csv")

## Update Databases

In [25]:
from app.utils.db_handler import DBHandler

db_handler = DBHandler()

In [26]:
db_handler.export_db_to_csv("api_voluum_ts_sources", "./data/sources.csv")

✅ Exported table using COPY to ./data/sources.csv


True

In [19]:
import pandas as pd

db = pd.read_csv("./data/monitor.csv")

In [20]:
db.head()

,custom_variable_1,ip,os_version,browser_version,timestamp_created,reason,source,click_2_reg,reg_2_ftd,clicks,conversions,cost,cost_sources,cpv,custom_conversions_1,custom_conversions_2,custom_conversions_3,custom_revenue_1,custom_revenue_2,custom_revenue_3,cv,epv,errors,profit,revenue,roi,suspicious_clicks,suspicious_clicks_percentage,suspicious_visits,suspicious_visits_percentage,unique_visits,visits
0,420728301905,185.131.242.34,Windows 11,Chrome 139,2025-12-29 19:49:24.701355,OS_VERSION,VOLUUM,0.0,0.0,0.0,0.0,1.31621,MANUAL_UPDATE,1.31621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.31621,0.0,-100.0,0.0,0.0,0.0,0.0,1.0,1.0
1,385914275551,148.113.211.163,Windows 10,Chrome 138,2025-12-29 19:49:24.701355,OS_VERSION,VOLUUM,0.0,0.0,0.0,0.0,1.08604,MANUAL_UPDATE,1.08604,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.08604,0.0,-100.0,0.0,0.0,1.0,100.0,1.0,1.0
2,61412709419,35.91.162.205,Windows 10,Chrome 113,2025-12-29 19:43:15.793203,OS_VERSION,VOLUUM,0.0,0.0,0.0,0.0,0.00000,NaN,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,1.0,100.0,1.0,1.0
3,61423475690,54.244.137.141,Windows 10,Chrome 113,2025-12-29 19:43:15.793203,OS_VERSION,VOLUUM,0.0,0.0,0.0,0.0,1.13706,MANUAL_UPDATE,1.13706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.13706,0.0,-100.0,0.0,0.0,1.0,100.0,1.0,1.0
4,38640557923,NaN,NaN,NaN,2026-01-16 14:02:14.435305,HLR_MONITORED,CSV_CLEAN_UPLOAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
def classifyRecord(osVersion):
    if pd.isna(osVersion):
        return "MONITOR"
    [os, version] = osVersion.split(' ') if osVersion.count(' ') == 1 else [osVersion, 'nil']
    majorVersion = int(version.split('.')[0]) if version != 'nil' and version[0].isdigit() else 9999
    # BLACKLIST rules first
    if os == 'IOS' and majorVersion <= 10:
        return 'BLACKLIST'   
    if os == 'Android' and majorVersion <= 7:
        return 'BLACKLIST'
    if osVersion in ['Windows 7', 'Windows XP'] or 'Ubuntu' in osVersion:
        return 'BLACKLIST'

    # MONITOR rules
    if os == 'Android' and majorVersion in [8, 9, 9999]:
        return 'MONITOR'
    if osVersion in ['Windows 10', 'Windows 11'] or os == 'Linux':
        return 'MONITOR'
    if os == 'Android' and 17 <= majorVersion:
        return "MONITOR"

    # WHITELIST rules
    if os == 'IOS' and 11 <= majorVersion:
        return 'WHITELIST'
    if 'MacOS' in os or 'Mac' in os:
        return 'WHITELIST'
    if os == 'Android' and 10 <= majorVersion < 17:
        return 'WHITELIST'  
    return 'UNKNOWN'

In [24]:
db["new_category"] = db["os_version"].apply(classifyRecord)
db["new_category"].value_counts()

new_category
MONITOR    44770
Name: count, dtype: int64